In [2]:
import socket,time

In [22]:
HOST = "10.107.11.59"
PORT = 6379

with open("auth") as auth:
    auth= auth.read()

with socket.socket(socket.AF_INET,socket.SOCK_STREAM) as s:
    s.connect((HOST,PORT))
    s.setsockopt(socket.IPPROTO_TCP,socket.TCP_NODELAY,1) 
    #The setsockopt() function provides an application program with the means to control socket behavior.
    #socket.IPPROTO_TCP is to specify the protocol to be used; tcp
    
    #time.sleep(2)
    s.sendall(f"auth {auth}\r\n".encode())
    
    print(s.recv(1024))
    
    s.sendall(b"ping\r\n")
    #if s.recv(1024) == b"+PONG\r\n":
    #    print("OK")
    print(s.recv(1024))
    


    

b'+OK\r\n'
b'+PONG\r\n'


In [32]:
def ClusterHealthCheck(agents):
    import socket
    success=[]
    for node in agents:
        print(type(node),node)
        with socket.socket(socket.AF_INET,socket.SOCK_STREAM) as sock:
            try:
                sock.connect(node) #tuple type
                sock.setsockopt(socket.IPPROTO_TCP,socket.TCP_NODELAY,1)
                if auth:
                    sock.sendall(f"auth {auth}\r\n".encode())
                    print(sock.recv(1024))
                sock.sendall(b"ping\r\n")
                if sock.recv(1024) == b"+PONG\r\n":
                    success.append(True)
                else:
                    success.append(False)
            except Exception as e:
                print(f"[ERROR] Connection to {node} failed.")
                success.append(False)
    if all(success):
        return True
    else:
        return False
    
agents=[("10.107.11.66",6379),("10.107.11.56",6379),("10.107.11.59",6379),("10.107.11.66",6380),("10.107.11.56",6380),("10.107.11.59",6380)]
print(len(agents))

ClusterHealthCheck(agents)

6
<class 'tuple'> ('10.107.11.66', 6379)
b'+OK\r\n'
<class 'tuple'> ('10.107.11.56', 6379)
b'+OK\r\n'
<class 'tuple'> ('10.107.11.59', 6379)
b'+OK\r\n'
<class 'tuple'> ('10.107.11.66', 6380)
b'+OK\r\n'
<class 'tuple'> ('10.107.11.56', 6380)
b'+OK\r\n'
<class 'tuple'> ('10.107.11.59', 6380)
[ERROR] Connection to ('10.107.11.59', 6380) failed.


False

#### AF_INET?

AF_INET is an address family that is used to designate the type of addresses that your socket can communicate with (in this case, ***Internet Protocol v4*** addresses).<br><br> 

When you create a socket, you have to specify its address family, and then you can only use addresses of that type with the socket.<br><br>

AF_INET's address format is basically host and port number whereas AF_UNIX's one is UNIX pathname.
<br><br>

For the most part, sticking with AF_INET for socket programming over a network is the safest option.<br>
There is also AF_INET6 for Internet Protocol v6 addresses.<br>


### SOCK_STREAM?
Socket type. default is set to TCP.<br>
In contrast, User Datagram Protocol(UDP) sockets created with socket.SOCK_DGRAM are not reliable<br>
and data read by the receiver can be out-of-order from sender's writes. <br><br>


### TCP_NODELAY
The TCP_NODELAY socket option allows your network to bypass "Nagle Delays" by disabling Nagle's algorithm, and sending the data as soon as it's available. Enabling TCP_NODELAY forces a socket to send the data in its buffer, whatever the packet size.

#### Nagle Delays?
This is means of improving the efficiency of TCP/IP networks by reducing the number of packets that need to be sent over the network. Reducing here means to store data and sent it once(buffering).



In [65]:
from itsdangerous import TimedJSONWebSignatureSerializer as Serializer
import requests

class Redis:
    def __init__(self,nodes,auth=None):
        self.nodes=nodes
        self.auth = auth
        self.agents=[]
        for node in self.nodes: #10.107.11.66:6379
            agent = node.split(":")
            self.agents.append((agent[0],int(agent[1])))
            "AGENT_KEY"
    def connector(self): #custom context manager?
        pass
    
        
    @staticmethod
    def token_generator() -> str:
        serializer= Serializer("AGENT_KEY",300)
        return serializer.dumps({"confirm":True}).decode("utf-8")
    
    def ClusterHealthCheck(self):
        import socket
        success=[]
        for node in self.agents:
            with socket.socket(socket.AF_INET,socket.SOCK_STREAM) as sock:
                try:
                    sock.connect(node) #tuple type
                    sock.setsockopt(socket.IPPROTO_TCP,socket.TCP_NODELAY,1)
                    if self.auth:
                        sock.sendall(f"auth {self.auth}\r\n".encode())
                        sock.recv(1024)
                    sock.sendall(b"ping\r\n")
                    if sock.recv(1024) == b"+PONG\r\n":
                        success.append(True)
                    else:
                        #print(f"[ERROR] Connection to {node} failed!")
                        success.append(False)
                except Exception as e:
                    #print(f"[ERROR] Connection to {node} failed!")
                    success.append(False)
        if all(success):
            return True
        else:
            return False
                    
        
    def RollingRestart(self):
        for node in self.agents:
            while True:
                try :
                    if self.ClusterHealthCheck():
                        print("Cluster health green! Continue rolling restart...")
                        token = Redis.token_generator()
                        res = requests.post("http://"+node[0]+":5000/redis/command/restart",json={"token":token,"port":node[1]})
                        if res.status_code==200:
                            print(f"[SUCCESS] Agent : {node} executed Restart...")
                            time.sleep(10) 
                        else:
                            print(f"[ERROR] Agent : {node} restart failed...")
                    else:
                        raise Exception(f"Connection to {node} failed!")
                except Exception as e:
                    print(f"[ERROR] {e}")
                    return False
                else: # If successful, 
                    cnt =0
                    MAXTRIAL= 5
                    while not self.ClusterHealthCheck() and cnt < MAXTRIAL:
                        
                        print("Cluster health not green! give it a little sec")
                        time.sleep(5)
                        cnt+=1
                        print(f"Tried {cnt} times...")
                        print(f"The most recent execution was on {node}")
                        continue
                    if not cnt < MAXTRIAL:
                        return False
                    else:
                        break
        return True

In [66]:
agents=["10.107.11.66:6379",#"10.107.11.56:6379","10.107.11.59:6379",
        "10.107.11.66:6380"]#"10.107.11.56:6380","10.107.11.59:6380"]



redis = Redis(agents,auth=1234)
redis.RollingRestart()

Cluster health green! Continue rolling restart...
[SUCCESS] Agent : ('10.107.11.66', 6379) executed Restart...
Cluster health green! Continue rolling restart...
[SUCCESS] Agent : ('10.107.11.66', 6380) executed Restart...


True